# Single facet simulation

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt

In [2]:
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant
from ibvpy.bcond import BCSlice, BCDof
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets import FETS3D8H
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from bmcs_matmod.slide.vslide_34 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic
from bmcs_utils.api import Model

In [5]:
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField

In [138]:
n_e = 1
L_x = 1.0
xd_left = XDomainFEGrid(coord_min=(-1, 0),
                          coord_max=(0, 1),
                          shape=(1, 1),
                          fets=FETS2D4Q())
xd_right = XDomainFEGrid(coord_min=(0, 0),
                          coord_max=(1, 1),
                          shape=(1, 1),
                          fets=FETS2D4Q())

In [139]:
m1 = MATS2DElastic(E=28000, nu=0.3)
m2 = MATS2DElastic(E=28000, nu=0.3)

xd12 = XDomainFEInterface(
    I=xd_left.mesh.I[-1, :],
    J=xd_right.mesh.I[0, :],
    fets=FETS1D52ULRH()
)

In [140]:
material_params = dict(
    E_T=10000, gamma_T=4000, K_T=0, S_T=0.01, c_T=2, bartau=5, 
    E_N=25000, S_N=0.05, c_N = 1, m = 0.008, f_t=5, f_c=150, f_c0=100, eta=0.0 # 0.5
)
bond_m = Slide34(**material_params)

In [168]:
m = TStepBC(
    domains=[
        (xd_left, m1),
        (xd_right, m2),
        (xd12, bond_m),
    ],
)

Note that the enumeration of `dofs` within a serialized `fe_domain` is done 
only after the domains have been registered within the `TStepBC`. Therefore,
final numbers of `dofs` are available only here and not before constructing `TStepBC`.
Whan manipulating the `dofs` individually - like here - when we want to 
construct nodal loads, the boundary conditions must be constructed once
the subdomains `xd_left`, `xd_right` and `xd12`  have been registered within the `TStepBC`.

Since `BCSlice` cannot be used currently with the 
natural boundary conditions we need to apply the nodal load to the individual degrees of  
freedom. To get them, let us misuse BCSlice as a quick access to the dofs. This can be 
done also directly by accessing the `mesh` object @RC - TODO

In [169]:
right_control_slice = BCSlice(slice=xd_right.mesh[:, :, :, :],
                              var='u', dims=[0], value=0)
right_control_dofs = [bc.dof for bc in right_control]
right_control_dofs

[8, 10, 12, 14]

Control boundary conditions are now defined as nodal values. Define the time function with on unloading cycle.

In [170]:
tf = TFCyclicNonsymmetricConstant(number_of_cycles=2)

To find out subcritical value of force, run first a displacement control simulation, find out the maximum value.
Then divide the obtained peak load with the number of control degrees of freedom, i.e. 4 in our case.

In [171]:
disp_control = False
if disp_control:
    right_control = [BCDof(var='u', dof=dof, value=0.02, time_function=tf ) for dof in right_control_slice.dofs ]
else:
    F_max = 50 / len(right_control_dofs)
    right_control = [BCDof(var='f', dof=dof, value=F_max, time_function=tf ) for dof in right_control_slice.dofs ]

Fix the remaining boundary conditions

In [172]:
right_fixed = BCSlice(slice=xd_right.mesh[:, :, :, :],
                 var='u', dims=[1], value=0)
left_fixed = BCSlice(slice=xd_left.mesh[:, :, :, :],
                 var='u', dims=[0,1], value=0)
bc1 = right_control + [right_fixed, left_fixed]

In [173]:
m.bc=bc1

In [174]:
m.hist.vis_record = {
    'strain': Vis3DTensorField(var='eps_ab'),
}
s = m.sim
s.tloop.verbose = True # set false to avoid the prints during each increment
s.tloop.k_max = 50
s.tline.step = 0.005

In [175]:
s.reset()
s.run()

t:	 0.00(0), 
	 0.01(1), 
	 0.01(1), 
	 0.01(1), 
	 0.02(1), 
	 0.03(1), 
	 0.03(27), 
	 0.04(27), 
	 0.04(27), 
	 0.04(27), 
	 0.05(27), 
	 0.05(27), 
	 0.06(27), 
	 0.06(27), 
	 0.07(27), 
	 0.07(27), 
	 0.08(27), 
	 0.09(27), 
	 0.09(27), 
	 0.10(27), 
	 0.10(27), 
	 0.11(28), 
	 0.11(28), 
	 0.12(28), 
	 0.12(28), 
	 0.13(28), 
	 0.13(28), 
	 0.14(28), 
	 0.14(28), 
	 0.15(29), 
	 0.15(29), 
	 0.16(29), 
	 0.16(29), 
	 0.17(29), 
	 0.17(30), 
	 0.18(30), 
	 0.18(30), 
	 0.19(31), 
	 0.19(31), 
	 0.20(31), 
	 0.20(32), 
	 0.21(32), 
	 0.21(33), 
	 0.22(33), 
	 0.22(34), 
	 0.23(34), 
	 0.23(35), 
	 0.24(36), 
	 0.24(37), 
	 0.25(38), 
	 0.25(39), 
	 0.26(1), 
	 0.26(1), 
	 0.27(1), 
	 0.27(1), 
	 0.28(1), 
	 0.28(1), 
	 0.29(1), 
	 0.29(1), 
	 0.30(1), 
	 0.30(1), 
	 0.31(1), 
	 0.31(1), 
	 0.32(1), 
	 0.32(1), 
	 0.33(1), 
	 0.33(1), 
	 0.34(1), 
	 0.34(1), 
	 0.35(21), 
	 0.35(24), 
	 0.36(24), 
	 0.36(24), 
	 0.37(24), 
	 0.37(24), 
	 0.38(25), 
	 0.38(25), 
	 0.39(25), 
	 0.39(2

RuntimeError: Factor is exactly singular

In [176]:
F_to = m.hist.F_t
U_to = m.hist.U_t
right_control_dofs = [bc.dof for bc in right_control]
F_l_t = np.sum(F_to[:, right_control_dofs], axis=-1)
U_r_t = np.average(U_to[:, right_control_dofs], axis=-1)
_, ax = plt.subplots(1,1)
ax.plot(U_r_t, F_l_t)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
if False:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'strain'
    strain_viz = Viz3DTensorField(vis3d=m.hist['strain'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()

In [14]:
m.fe_domain[2].state_n

{'w_pi': array([[8.73010605e-05, 8.73010605e-05]]),
 's_pi_x': array([[-0.01612612, -0.01612612]]),
 's_pi_y': array([[0., 0.]]),
 'z': array([[0.00873011, 0.00873011]]),
 'alpha_x': array([[-0.00873011, -0.00873011]]),
 'alpha_y': array([[0., 0.]]),
 'omega_T': array([[0.73568455, 0.73568455]]),
 'omega_N': array([[0., 0.]]),
 'sig_pi': array([[0., 0.]]),
 'tau_pi_x': array([[0., 0.]]),
 'tau_pi_y': array([[0., 0.]]),
 'Z': array([[0., 0.]]),
 'X_x': array([[0., 0.]]),
 'X_y': array([[0., 0.]]),
 'Y_T': array([[0., 0.]]),
 'Y_N': array([[0., 0.]])}